In [3]:
!pip install nufhe[pycuda,pyopencl]

     |████████████████████████████████| 919 kB 1.5 MB/s eta 0:00:01
time: 3.11 s (started: 2023-03-22 10:23:24 +05:30)


In [ ]:
!pip install ipython-autotime

In [1]:

%load_ext autotime
nvidia-smi

NameError: name 'nvidia' is not defined

time: 314 ms (started: 2023-03-24 08:59:45 +05:30)


In [2]:
import random
import nufhe
size=8
def fixSizeBoolList(decimal,size):
    x = [[int(x)] for x in bin(decimal)[2:]]
    for i in range(len(x)):
        x[i] = list(map(bool, x[i]))
    x = [[False]]*(size - len(x)) + x
    x.reverse()
    return x
bits1 = fixSizeBoolList(2,size)
ctx = nufhe.Context()
secret_key, cloud_key = ctx.make_key_pair()

ciphertext1 = ctx.encrypt(secret_key, bits1)
print(bits1)   

[[False], [True], [False], [False], [False], [False], [False], [False]]
time: 1.85 s (started: 2023-03-24 08:59:54 +05:30)


In [3]:
import random
import nufhe
def add(ciX, ciY):
# fixed iteration since
    for i in range(size):
        a = vm.gate_and(ciX, ciY)
        b = vm.gate_xor(ciX, ciY)
        aShiftTemp = a
        # using roll as a shift bit
        aShiftTemp.roll(1, axis=0)
        ciX = aShiftTemp
        ciY = b

    return b

def boolListToInt(bitlists):
    out = 0
    bitlists[:] = bitlists[::-1]
    for bit in bitlists:
        out = (out << 1) | bit
    return out

def fixSizeBoolList(decimal,size):
    x = [[int(x)] for x in bin(decimal)[2:]]
    for i in range(len(x)):
        x[i] = list(map(bool, x[i]))
    x = [[False]]*(size - len(x)) + x
    x.reverse()
    return x
size = 8
#bits1 = [random.choice([False, True]) for i in range(size)]
#bits2 = [random.choice([False, True]) for i in range(size)]
bits1 = fixSizeBoolList(10,size)
bits2 = fixSizeBoolList(2,size)
s=[ [False] for i in range(size)]
reference = [not (b1 and b2) for b1, b2 in zip(bits1, bits2)]

ctx = nufhe.Context()
secret_key, cloud_key = ctx.make_key_pair()

ciphertext1 = ctx.encrypt(secret_key, bits1)
ciphertext2 = ctx.encrypt(secret_key, bits2)
c_s=ctx.encrypt(secret_key, s)

vm = ctx.make_virtual_machine(cloud_key)
print(bits1)
print(bits2)
c_s[0]=ciphertext2[0]
result = add(ciphertext1, ciphertext2)
print("=========")
print(s)
print("type iss",type(c_s[0]))
print("=========================")
#result=vm.gate_mux(c_s[0],ciphertext1, ciphertext2)
result_bits = ctx.decrypt(secret_key, result)

print(result_bits)
print("result is",boolListToInt(result_bits))
#assert all(result_bits == reference)

[[False], [True], [False], [True], [False], [False], [False], [False]]
[[False], [True], [False], [False], [False], [False], [False], [False]]
[[False], [False], [False], [False], [False], [False], [False], [False]]
type iss <class 'nufhe.lwe.LweSampleArray'>
[[False]
 [False]
 [ True]
 [ True]
 [False]
 [False]
 [False]
 [False]]
result is [12]
time: 3.53 s (started: 2023-03-24 08:59:57 +05:30)


In [4]:
import random
import nufhe
import time
import numpy as np
ctx = nufhe.Context()
secret_key, cloud_key = ctx.make_key_pair()
vm = ctx.make_virtual_machine(cloud_key)
size = 16

time: 1.34 s (started: 2023-03-24 09:00:03 +05:30)


In [5]:
#from google.colab import drive
#drive.mount('/content/drive')
#with open('/content/drive/MyDrive/pc11.txt') as f:


time: 606 µs (started: 2023-03-24 09:00:05 +05:30)


In [6]:
def add(ciX, ciY):
# fixed iteration since
    
    for i in range(size):
        a = vm.gate_and(ciX, ciY)
        b = vm.gate_xor(ciX, ciY)
        aShiftTemp = a
        # using roll as a shift bit
        aShiftTemp.roll(1, axis=0)
        #aShiftTemp=np.roll(aShiftTemp,-1, axis=0)
        ciX = aShiftTemp
        ciY = b
    #result=ctx.decrypt(secret_key,b)
    #print("result is",boolListToInt(result))
    return b
'''def boolListToInt(bitlists):
    out = 0
    bitlists[:] = bitlists[::-1]
    for bit in bitlists:
        out = (out << 1) | bit
    return out'''
def boolListToInt(bitlists):
    out = 0
    for bit in bitlists:
        out = (out << 1) | bit
    return out
'''def fixSizeBoolList(decimal,size):
    x = [int(x) for x in bin(decimal)[2:]]
    x = list(map(bool, x))
    x = [False]*(size - len(x)) + x
    x.reverse()
    return x'''
def fixSizeBoolList(decimal,size):
    x = [[int(x)] for x in bin(decimal)[2:]]
    for i in range(len(x)):
        x[i] = list(map(bool, x[i]))
    x = [[False]]*(size - len(x)) + x
    x.reverse()
    return x





def subtract(ciX, ciY):
    for i in range(size):
        ciXnotTemp = ciX
        a = vm.gate_and(vm.gate_not(ciX), ciY)
        ciX = vm.gate_xor(ciX, ciY)
        aShiftTemp = a
        aShiftTemp.roll(-1, axis=-1)
        ciY = aShiftTemp
    return ciX
def equal_checkprint(ciX, ciY):
    result=subtract(ciX,ciY)
    result1=subtract(ciY,ciX)
    result2=vm.gate_or(result,result1)
    for i in range(size):
        aShiftTemp=result2
        aShiftTemp.roll(-1, axis=-1)
        shift = aShiftTemp
    not_result=vm.gate_not(shift)
    return not_result
def greater(ciX, ciY):
    result=subtract(ciX,ciY)
    for i in range(size):
        abc.a[i]=result.a[0]
        abc.b[i]=result.b[0]
    return abc
def lesser(ciX, ciY):
    result=subtract(ciX,ciY)
    for i in range(size):
        abc.a[i]=result.a[0]
        abc.b[i]=result.b[0]
    return abc
def scalarEncrypt(X):
      x=fixSizeBoolList(X,size)
      #px=np.zeros((size), dtype=bool)
      #px=np.concatenate([px,x])
      ciX = ctx.encrypt(secret_key, X)
      return ciX
def scalarDecrypt(X):
    result_scalar=[]
    for i in range (size+1):
      y = ctx.decrypt(secret_key,X[i])
      result_scalar.append(y[size:],size)
def shift(x,enc_temp):
  
  zero= [False]
  enc_zero=vm.gate_constant(zero)
  for i in range(size-1,0,-1):
     enc_temp[i]=x[i-1]
  enc_temp[0]=enc_zero   
  return enc_temp
def mux(c_s,true,false) :
    temp1=vm.gate_and(c_s,true)
    #print("temp1:",ctx.decrypt(secret_key, temp1))
    temp2=vm.gate_and(vm.gate_not(c_s),false)
    #print("temp2:",ctx.decrypt(secret_key, temp2))
    mux_result=vm.gate_or(temp1,temp2)
    return mux_result
def mul(ciX,ciY,enc_temp):
    
    msb_temp= [vm.empty_ciphertext((1,))]
    control=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    c_s=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    c_s1=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    msb_temp=vm.gate_xor(ciX[size-1],ciY[size-1])
    t=size-1
    for i in range(size):
            control[i]=ciX[t]
            c_s1[i]=ciY[t]
    mul_temp=ciX
    zero=enc_temp
    one_ciX=vm.gate_not(ciX)
    result=ctx.decrypt(secret_key, one_ciX)
    one_ciY=vm.gate_not(ciY)
    comp_ciX=add(one_ciX,one)
    comp_ciY=add(one_ciY,one)
    
    #print("control bit:",ctx.decrypt(secret_key, control))
    ciX=mux(control,comp_ciX,ciX)
    ciY=mux(c_s1,comp_ciY,ciY)
    #print("cix:",ctx.decrypt(secret_key, ciX))
    for j in range(0,size):
        #print("=================iteration==========",j)
        
        
        #print("cix:",ctx.decrypt(secret_key, ciX))
        for i in range(size):
            c_s[i]=ciY[j]
        add_res=mux(c_s,ciX,zero)
        
        enc_theta=add(enc_temp,add_res)
        enc_temp=enc_theta
        #enc_temp=ciX
        #print("enc_teta:",ctx.decrypt(secret_key, enc_theta))
        
        #print("control bit:",ctx.decrypt(secret_key, c_s[0]))
        #mux_result=mux(c_s,enc_theta,ciX)
        #mux_result=vm.gate_mux(c_s, enc_theta,ciX)
        #print("mux result:",ctx.decrypt(secret_key, mux_result))
        #ciX=mux_result
        ciX.roll(1, axis=0)
        #ciX.roll(1, axis=0)
    for i in range(size):
        c_s[i]=msb_temp    
    one_presult=vm.gate_not(enc_theta)
    comp_presult=add(one_presult,one)
    ciX=vm.gate_mux(c_s,comp_presult,enc_theta)
    return ciX



'''#works only for positive numbers
def mul(ciX,ciY,enc_temp):
    mul_temp=ciX
    zero=enc_temp
    for j in range(0,size):
        #print("=================iteration==========",j)
        
        
        #print("cix:",ctx.decrypt(secret_key, ciX))
        for i in range(size):
            c_s[i]=ciY[j]
        add_res=mux(c_s,ciX,zero)
        
        enc_theta=add(enc_temp,add_res)
        enc_temp=enc_theta
        #enc_temp=ciX
        #print("enc_teta:",ctx.decrypt(secret_key, enc_theta))
        
        #print("control bit:",ctx.decrypt(secret_key, c_s[0]))
        #mux_result=mux(c_s,enc_theta,ciX)
        #mux_result=vm.gate_mux(c_s, enc_theta,ciX)
        #print("mux result:",ctx.decrypt(secret_key, mux_result))
        #ciX=mux_result
        ciX.roll(1, axis=0)
        #ciX.roll(1, axis=0)
    return enc_theta'''  
def predict(ctA,onep,onen):
    c_s=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    for i in range(size):
        c_s[i]=ctA[size-1]
        
    res= vm.gate_mux(c_s,onen,onep)
    
    
    return res
def Convert_list(string):
    list1=[]
    list1[:0]=string
    #print(list1)
    list1=[int(i)for i in list1 ]
    listb=[]
    for i in list1:
        if i==0:
            listb.append([False])
        else:
            listb.append([True])    
    
    #print(listb)
    return listb
def twos_complement(n,nbits):
    a=f"{n & ((1 << nbits) - 1):0{nbits}b}" 
    #print(type(a))
    a=Convert_list(a)
    a.reverse()
    return a
def listToString(s): 
    # initialize an empty string
    list1=[int(i)for i in s ]
    listp=[]
    for i in list1:
        if i==False:
            listp.append('0')
        else:
            listp.append('1')    
    
    #print(listp)
    str1 = "" 
    # traverse in the string  
    s=['delim'.join([str(elem) for elem in sublist]) for sublist in listp]
    #print(s)
    for ele in s: 
        str1 += ele  
    # return string 
    #print(str1)
    return str1
def two2dec(s):
    if s[0] == '1':
        return -1 * (int(''.join('1' if x == '0' else '0' for x in s), 2) + 1)
    else:
        return int(s, 2)

time: 3.53 ms (started: 2023-03-24 09:00:07 +05:30)


In [8]:
# Equality check using NUFHE
import numpy as np
import time
with open('/home/pradeep/Downloads/pc11.txt') as f:
    lines = []
    for line in f:
        lines.append([int(v) for v in line.split()])
w_p=[-4, 6, -1, -1, 3, 1, 1, 2, 2, -4, 1, 6]
zero= np.zeros((size,1), dtype=bool)
A = fixSizeBoolList(7,size)
B = fixSizeBoolList(9,size)
s=[ [False] for i in range(size)]
test_size = int(input("Please enter a string:\n"))
theta = fixSizeBoolList(0,size)
temp=fixSizeBoolList(0,size)
Y = fixSizeBoolList(7,size)
one=ctx.encrypt(secret_key, twos_complement(1,size))
ciX=ctx.encrypt(secret_key, A)
ciY=ctx.encrypt(secret_key, B)
c_s=ctx.encrypt(secret_key, s)
#onen=  [ctx.encrypt(secret_key, [True]) for _ in range(size)]        
#onep=  [ctx.encrypt(secret_key, [False]) for _ in range(size)]
print("==============")

print("cix:",ctx.decrypt(secret_key, ciX))

#enc_theta=ctx.encrypt(secret_key, theta)
enc_temp=ctx.encrypt(secret_key, temp)
enc_temp1=enc_temp
#print("type of cx is",type(ciX[0]))
#print("type is",type(enc_temp[0]))
print("==============")
enc_val=ctx.encrypt(secret_key, Y)
abc=vm.gate_constant(zero)
'''begin = time.time()
#not_result=equal_check(ciX, ciY)
mul_temp=ciX
for j in range(0,size-1):
    print("=================iteration==========",j)
    ciX=ciX
    print("cix:",ctx.decrypt(secret_key, ciX))
  
    enc_theta=add(ciX,mul_temp)
    print("enc_teta:",ctx.decrypt(secret_key, enc_theta))
    for i in range(size):
        c_s[i]=ciY[j] 
    print("control bit:",ctx.decrypt(secret_key, c_s[0]))
    mux_result=mux(c_s,enc_theta,ciX)
    #mux_result=vm.gate_mux(c_s, enc_theta,ciX)
    print("mux result:",ctx.decrypt(secret_key, mux_result))
    ciX=mux_result
end = time.time()'''

n=twos_complement(-1,size)
p=twos_complement(1,size)
onen=ctx.encrypt(secret_key, n)
onep=ctx.encrypt(secret_key, p)
w_x0=twos_complement(w_p[0],size)
print("w_x1  is",w_x0)
w_x1=twos_complement(w_p[1],size)

w_x2=twos_complement(w_p[2],size)
w_x3=twos_complement(w_p[3],size)
w_x4=twos_complement(w_p[4],size)
w_x5=twos_complement(w_p[5],size)
print("5 and 6 is",w_p[5],w_p[6])
w_x6=twos_complement(w_p[6],size)
w_x7=twos_complement(w_p[7],size)
w_x8=twos_complement(w_p[8],size)
w_x9=twos_complement(w_p[9],size)
w_x10=twos_complement(w_p[10],size)
bias_x0=twos_complement(w_p[11],size)

w_ciphertext1=ctx.encrypt(secret_key, w_x0)
print("ciy:",ctx.decrypt(secret_key, w_ciphertext1))
w_ciphertext2=ctx.encrypt(secret_key, w_x1)
w_ciphertext3=ctx.encrypt(secret_key, w_x2)
w_ciphertext4=ctx.encrypt(secret_key, w_x3)
w_ciphertext5=ctx.encrypt(secret_key, w_x4)
w_ciphertext6=ctx.encrypt(secret_key, w_x5)
w_ciphertext7=ctx.encrypt(secret_key, w_x6)
w_ciphertext8=ctx.encrypt(secret_key, w_x7)
w_ciphertext9=ctx.encrypt(secret_key, w_x8)
w_ciphertext10=ctx.encrypt(secret_key, w_x9)
w_ciphertext11=ctx.encrypt(secret_key, w_x10)
bias_ciphertext=ctx.encrypt(secret_key, bias_x0)
begin = time.time()
for i in range(test_size):
    print(lines[i][0],lines[i][1],lines[i][2],lines[i][3],lines[i][4],lines[i][5],lines[i][6],lines[i][7],lines[i][8],lines[i][9],lines[i][10])
    temp=int(lines[i][0])
    b_x0=twos_complement(temp,size)
    temp=int(lines[i][1])
    b_x1=twos_complement(temp,size)
    temp=int(lines[i][2])
    b_x2=twos_complement(temp,size)
    temp=int(lines[i][3])
    b_x3=twos_complement(temp,size)
    temp=int(lines[i][4])
    b_x4=twos_complement(temp,size)
    temp=int(lines[i][5])
    b_x5=twos_complement(temp,size)
    temp=int(lines[i][6])
    b_x6=twos_complement(temp,size)
    temp=int(lines[i][7])
    b_x7=twos_complement(temp,size)
    temp=int(lines[i][8])
    b_x8=twos_complement(temp,size)
    temp=int(lines[i][9])
    b_x9=twos_complement(temp,size)
    temp=int(lines[i][10])
    b_x10=twos_complement(temp,size)
    ciphertext1=ctx.encrypt(secret_key, b_x0)
    ciphertext2=ctx.encrypt(secret_key, b_x1)
    ciphertext3=ctx.encrypt(secret_key, b_x2)
    ciphertext4=ctx.encrypt(secret_key, b_x3)
    ciphertext5=ctx.encrypt(secret_key, b_x4)
    ciphertext6=ctx.encrypt(secret_key, b_x5)
    ciphertext7=ctx.encrypt(secret_key, b_x6)
    ciphertext8=ctx.encrypt(secret_key, b_x7)
    ciphertext9=ctx.encrypt(secret_key, b_x8)
    ciphertext10=ctx.encrypt(secret_key, b_x9)
    ciphertext11=ctx.encrypt(secret_key, b_x10)
    presult_mul1 = mul(ciphertext1,w_ciphertext1,enc_temp)
    res=ctx.decrypt(secret_key, presult_mul1)
    val=listToString(res)
    print("partialpredict1mul ",two2dec(val[::-1]) )
    presult_mul2 = mul(ciphertext2,w_ciphertext2,enc_temp)
    res=ctx.decrypt(secret_key, presult_mul2)
    val=listToString(res)
    print("partialpredict2mul ",two2dec(val[::-1]) )
    presult_mul3 = mul(ciphertext3,w_ciphertext3,enc_temp)
    res=ctx.decrypt(secret_key, presult_mul3)
    val=listToString(res)
    print("partialpredict3mul ",two2dec(val[::-1]) )
    presult_mul4 = mul(ciphertext4,w_ciphertext4,enc_temp)
    res=ctx.decrypt(secret_key,presult_mul4)
    val=listToString(res)
    print("partialpredict4mul ",two2dec(val[::-1]) )
    presult_mul5 = mul(ciphertext5,w_ciphertext5,enc_temp)
    res=ctx.decrypt(secret_key, presult_mul5)
    val=listToString(res)
    print("partialpredict5mul ",two2dec(val[::-1]) )
    presult_mul6 = mul(ciphertext6,w_ciphertext6,enc_temp)
    res=ctx.decrypt(secret_key, presult_mul6)
    val=listToString(res)
    print("partialpredict6mul ",two2dec(val[::-1]) )
    presult_mul7 = mul(ciphertext7,w_ciphertext7,enc_temp)
    res=ctx.decrypt(secret_key, presult_mul7)
    val=listToString(res)
    print("partialpredict7mul ",two2dec(val[::-1]) )
    presult_mul8 = mul(ciphertext8,w_ciphertext8,enc_temp)
    res=ctx.decrypt(secret_key, presult_mul8)
    val=listToString(res)
    print("partialpredict8mul ",two2dec(val[::-1]) )
    presult_mul9 = mul(ciphertext9,w_ciphertext9,enc_temp)
    res=ctx.decrypt(secret_key, presult_mul9)
    val=listToString(res)
    print("partialpredict9mul ",two2dec(val[::-1]) )
    presult_mul10 = mul(ciphertext10,w_ciphertext10,enc_temp)
    res=ctx.decrypt(secret_key,presult_mul10)
    val=listToString(res)
    print("partialpredict10mul ",two2dec(val[::-1]) )
    presult_mul11 = mul(ciphertext11,w_ciphertext11,enc_temp)
    res=ctx.decrypt(secret_key, presult_mul11)
    val=listToString(res)
    print("partialpredict11mul ",two2dec(val[::-1]) )
    presult_add1 = add(presult_mul1, presult_mul2)
    res=ctx.decrypt(secret_key, presult_add1)
    val=listToString(res)
    print("partialpredictadd1 ",two2dec(val[::-1]) )
    presult_add2 = add(presult_mul3, presult_mul4)
    res=ctx.decrypt(secret_key, presult_add2)
    val=listToString(res)
    print("partialpredictadd2 ",two2dec(val[::-1]) )
    presult_add3 = add(presult_mul5, presult_mul6)
    res=ctx.decrypt(secret_key, presult_add3)
    val=listToString(res)
    print("partialpredictadd3 ",two2dec(val[::-1]) )
    presult_add4 = add(presult_mul7, presult_mul8)
    res=ctx.decrypt(secret_key, presult_add4)
    val=listToString(res)
    print("partialpredictadd4 ",two2dec(val[::-1]) )
    presult_add5 = add(presult_mul9, presult_mul10)
    res=ctx.decrypt(secret_key, presult_add5)
    val=listToString(res)
    print("partialpredictadd5 ",two2dec(val[::-1]) )
    #first step
    presult_add11 = add(presult_add1, presult_add2)
    presult_add12 = add(presult_add3, presult_add4)
    presult_add13 = add(presult_add5, presult_mul11)
    #secong step
    presult_add21 = add(presult_add11, presult_add12)
    presult_add22 = add(presult_add13, bias_ciphertext)
    partial_predict = add(presult_add21,presult_add22)
    res=ctx.decrypt(secret_key, partial_predict)
    val=listToString(res)
    print("partialpredict ",two2dec(val[::-1]) )
    predict1=predict(partial_predict,onep,onen)
    res=ctx.decrypt(secret_key, predict1)
    val=listToString(res)
    print("2scomplement to dec ",two2dec(val[::-1]) )
    #print("final result is",boolListToInt(res))
end = time.time()
print(f"Total runtime for 32-bit full adder is {float(end - begin)} second")

#print(f"Total runtime for 32-bit full adder is {float(end - begin)} second")
#assert all(result_bits == reference)
#ciX=mul(ciX,ciY,enc_temp1)

#result=ctx.decrypt(secret_key, ciX)
#print("mux result:",)
#print("final result is",boolListToInt(result))

Please enter a string:
5
cix: [[ True]
 [ True]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]]
w_x1  is [[False], [False], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True]]
5 and 6 is 1 1
ciy: [[False]
 [False]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]]
81 4 -25 -26 -13 14 -17 -5 7 5 7
partialpredict1mul  -324
partialpredict2mul  24
partialpredict3mul  25
partialpredict4mul  26
partialpredict5mul  -39
partialpredict6mul  14
partialpredict7mul  -17
partialpredict8mul  -10
partialpredict9mul  14
partialpredict10mul  -20
partialpredict11mul  7
partialpredictadd1  -300
partialpredictadd2  51
partialpredictadd3  -25
partialpredictadd4  -26
partialpredictadd5  -6
partialpredict  -292
2scomplement to dec  -1
1 -28 -1 -14 1 0 -3 -4 -7 1 4
partialpredict1mul  -4
partialpre

In [ ]:
# Find out the Greater number 
zero= np.zeros((size,), dtype=bool)
A = fixSizeBoolList(4,size)
B = fixSizeBoolList(6,size)
#abc=vm.gate_constant(zero)
ciX=ctx.encrypt(secret_key, A)
ciY=ctx.encrypt(secret_key, B)
begin = time.time()
abc=greater(ciX, ciY)
And_result=vm.gate_and(shift,enc_theta)
mux_result=vm.gate_mux(abc, ciY, ciX)
end = time.time()
print(f"Total runtime for 32-bit full adder is {float(end - begin)} second")
print("Abc:",ctx.decrypt(secret_key, abc))
print("mux result:",ctx.decrypt(secret_key, mux_result))

In [ ]:
# Find out the Lesser number 
zero= np.zeros((size,), dtype=bool)
A = fixSizeBoolList(4,size)
B = fixSizeBoolList(6,size)
abc=vm.gate_constant(zero)
ciX=ctx.encrypt(secret_key, A)
ciY=ctx.encrypt(secret_key, B)
begin = time.time()
abc=lesser(ciX, ciY)
mux_result=vm.gate_mux(abc, ciX, ciY)
end = time.time()
print(f"Total runtime for 32-bit full adder is {float(end - begin)} second")
#assert all(result_bits == reference)


print("Abc:",ctx.decrypt(secret_key, abc))
print("mux result:",ctx.decrypt(secret_key, mux_result))

In [ ]:
!wget https://drive.google.com/drive/u/1/my-drive/dataset.txt

In [ ]:
import numpy as np
example1 = "/content/dataset.txt"
file1 = open(example1, "r")

In [ ]:
FileContent = file1.read()
FileContent
print(FileContent)
file1.close()

In [ ]:
with open(example1, "r") as file1:
    FileasList = file1.readlines()
   

In [ ]:
 FileasList[0]

In [ ]:
FileasList = [i.split('\t')[0] for i in FileasList] 

print(FileasList)


In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_csv("pima-indians-diabetes.csv", sep=",", header=None)
df = df.iloc[1: , :]

In [ ]:
#Read a file using pandas and store it in a matrix
#convert all elements in matrix into integer using astype() 
df.columns = ["A", "B", "C","D", "E", "F","G", "H", "I" ]
#print(df.head)
#print(df)
mat = np.array(df)
mat = mat.astype(float)
mat = mat.astype(int)
#print("matrix : \n")
#print(mat[0][0])
#print(len(mat))

#for i in range(len(mat)):
#  for j in range(len(mat[0])):
 #   print(mat[i][j], end=" ")
  #print("\n")
x1=[]
matrix=mat.flatten(order='C')
print(matrix)
x=[5,2,12,9,4]
print(x[0])
a=fixSizeBoolList(x[0],size)
b=fixSizeBoolList(2,size)
c=fixSizeBoolList(12,size)
d=fixSizeBoolList(9,size)
e=fixSizeBoolList(4,size)
ci_a=ctx.encrypt(secret_key, a)
ci_b=ctx.encrypt(secret_key, b)
ci_c=ctx.encrypt(secret_key, c)
ci_d=ctx.encrypt(secret_key, d)
ci_e=ctx.encrypt(secret_key, e)
print("a:",ctx.decrypt(secret_key, a))
print("b:",ctx.decrypt(secret_key, b))
print("c:",ctx.decrypt(secret_key, c))
print("d:",ctx.decrypt(secret_key, d))
print("e:",ctx.decrypt(secret_key, e))

#for i in range(len(x)):
#  enc_sample=scalarEncrypt(x[i])
 # x1[i]=enc_sample
#enc_dec=scalarDecrypt(enc_sample)
#print(enc_dec)
#ciX=[]##np.full_like(mat,enc_theta)
  #print("\n")
#for i in range(len(x)):
 # for j in range(len(mat[0])):
  #  mat_val=mat[i][j]
 #  ciX.append(ctx.encrypt(secret_key, x))

   